In [ ]:
from tqdm import tqdm
import os
from datasets import load_dataset
import json
import nltk
import re
import difflib
from copy import deepcopy

import json
import numpy as np
import random
import pandas as pd
from ollama import AsyncClient

: 

In [20]:
client_ollama = AsyncClient(
  host='http://192.168.1.5:11434',
  headers={'x-some-header': 'some-value'}
)

async def chat(content,image_path):
    message = {'role': 'user', 'content': content, 'images': [image_path]}
    response = await client_ollama.chat(model='llama3.2-vision', messages=[message])
    return response['message']['content']

In [31]:
task = 'EPPO'
aids = load_dataset("parquet", data_files={'validation': '/workdir/AGRIVQA/'+task+'/validation-*.parquet',
                                               'dev': '/workdir/AGRIVQA/'+task+'/dev-*.parquet',
                                               'test': '/workdir/AGRIVQA/'+task+'/test-*.parquet'})

In [32]:
aids

DatasetDict({
    validation: Dataset({
        features: ['id', 'question', 'options', 'explanation', 'image_1', 'image_2', 'image_3', 'image_4', 'image_5', 'img_type', 'answer', 'options_difficulty', 'question_type', 'subfield', 'metadata'],
        num_rows: 16519
    })
    dev: Dataset({
        features: ['id', 'question', 'options', 'explanation', 'image_1', 'image_2', 'image_3', 'image_4', 'image_5', 'img_type', 'answer', 'options_difficulty', 'question_type', 'subfield', 'metadata'],
        num_rows: 2064
    })
    test: Dataset({
        features: ['id', 'question', 'options', 'explanation', 'image_1', 'image_2', 'image_3', 'image_4', 'image_5', 'img_type', 'answer', 'options_difficulty', 'question_type', 'subfield', 'metadata'],
        num_rows: 2065
    })
})

In [35]:
aids['test'][0]['image_1'].save('/workdir/images/cache/prova.png', format='png')
question = aids['test'][0]['question']
options = aids['test'][0]['options']
print(await chat(question+' '+options,'/workdir/images/cache/prova.png'))

The image shows a Crocosmia flower, which is commonly known as an autumn crocus or montbretia. It is not a maiden hair tree, autumn bugbane, Martinez's spruce, or any other plant species.

It is important to note that the correct common name for this plant is "montbretia" and not "autumn crocus", which is actually a different type of flower. The montbretia is a popular choice for gardens due to its vibrant orange-red flowers and ability to attract pollinators like bees and butterflies.

It's also worth mentioning that the scientific name for this plant is Crocosmia x crocosmiiflora, which indicates that it is a hybrid species derived from two different Crocosmia species. This hybridization has resulted in the creation of a unique and attractive flower that is well-suited to various environments.


In [34]:
aids['test'][0]

{'id': 'test__EPPO_1',
 'question': 'What is the common name in English of this plant? <image 1>',
 'options': '[\'maidenhair tree\', \'autumn bugbane\', "martinez\'s spruce", \'montbretia\']',
 'explanation': None,
 'image_1': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x682>,
 'image_2': None,
 'image_3': None,
 'image_4': None,
 'image_5': None,
 'img_type': "['Picture']",
 'answer': 'D',
 'options_difficulty': '1',
 'question_type': 'multiple-choice',
 'subfield': 'Plantae / common_name',
 'metadata': '{"source": "EPPO", "author": "EPPO", "license": "", "url": "https://gd.eppo.int/media/data/taxon/T/TTRCR/pics/1024x0/8275.jpg", "language": "English", "verbose_answer": "{verbose_answer}", "eppo_code": "TTRCR", "gbif_key": "2747939", "kingdom": "Plantae", "tag": "common_name", "common_name_language": "English"}'}

In [9]:
for row in aids['test']:
    print(type(row['image_1']))
    

<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>
